# Shortest Path Experiments

In this notebook we run experiments on learning to optimize over shortest path routes.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import gurobipy as gp 
import numpy as np 
import networkx as nx
import pickle 
import time

from reweighted_mse_helpers import *
from sklearn.ensemble import RandomForestRegressor
from joblib import Parallel, delayed
from sklearn.linear_model import LinearRegression
from scipy.sparse import coo_matrix,csr_matrix

from models import LinearModel
from sgd import SGDLearner
import pandas as pd

## Shortest Path Optimization Oracle

We start by defining an oracle class to solve shortest path instances. This could probably be sped up by substituting an implementation of djikstra or other quick shortest path solvers.

In [2]:
class ShortestPathOracle(object):
    
    def __init__(self, graph_params,  quiet=True):
        
        nodes = graph_params['nodes']
        sources = graph_params['sources']
        destinations = graph_params['destinations']
        start_node = graph_params['start_node']
        end_node = graph_params['end_node']

        
        n_nodes = len(nodes)
        n_edges = len(sources)
        d_feasibleregion = len(sources)
        
        # Hard code sparse node-edge incidence matrix!
        I_vec = np.hstack([sources,destinations])
        J_vec = np.hstack([range(n_edges),range(n_edges)])
        V_vec = np.hstack([-np.ones(n_edges), np.ones(n_edges)])
        
        #Construct constraint matrix
        A_mat = coo_matrix((V_vec, (I_vec, J_vec)))
        A_mat = csr_matrix(A_mat)
        bvec = np.zeros(n_nodes)
        bvec[start_node] = -1; bvec[end_node] = 1; 
        
        A_mat = csr_matrix(A_mat)
        bvec = np.zeros(n_nodes)
        bvec[start_node] = -1; bvec[end_node] = 1; 
        self.m = gp.Model()
        if quiet: self.m.setParam("OutputFlag", 0)
        self.w = self.m.addMVar(n_edges, lb= 0, ub = 1)
        self.m.addConstrs(A_mat[i,:] @ self.w == bvec[i] for i in range(n_nodes) )
        
    def init_model(self,params):
        pass
    
    def solve(self, c):
        self.m.setObjective( c @ self.w, gp.GRB.MINIMIZE)
        self.m.optimize()
        z_ast = self.m.objVal
        w_ast = np.asarray([self.w[i].X for i in range(len(c))]).flatten()
        return [z_ast, w_ast]

From the SPO paper: 
In the following set of experiments on the shortest path problem we described, we fix the number of features at p = 5 throughout and, as previously mentioned, use a 5 × 5 grid network, which implies that d = 40. Hence, in total there are pd = 200 parameters to estimate.

In [3]:
[sources, destinations, scalar_nodes, tuple_nodes] = convert_grid_to_list(5, 5)
grid_dim = 5
nodes = np.unique(list(set(sources).union(set(destinations))))
d_feasibleregion = len(sources)
start_node = scalar_nodes[(0,0)]
end_node = scalar_nodes[(4,4)]

#We'll save these parameters and pass them on to each worker to generate the oracle
graph_params = {
    'nodes': nodes,
    'sources': sources,
    'destinations': destinations,
    'start_node': start_node,
    'end_node': end_node
}

{(0, 0): 0, (0, 1): 1, (0, 2): 2, (0, 3): 3, (0, 4): 4, (1, 0): 5, (1, 1): 6, (1, 2): 7, (1, 3): 8, (1, 4): 9, (2, 0): 10, (2, 1): 11, (2, 2): 12, (2, 3): 13, (2, 4): 14, (3, 0): 15, (3, 1): 16, (3, 2): 17, (3, 3): 18, (3, 4): 19, (4, 0): 20, (4, 1): 21, (4, 2): 22, (4, 3): 23, (4, 4): 24}


## Data Generator 

We follow the set-up of Elmachtoub & Grigas (2017) and generate data using a polynomial kernal with some noise.

#### Data Generating Parameters

In [4]:
# Generate Data
p_features=5
B_true = rand(1,0.5,size=(d_feasibleregion, p_features))
polykernel_noise_half_width = 0.5

## Experiment

#### Experiment Parameters

In [5]:
#Amount of training data
N_s = [100, 1000, 2000]

#degrees of mis-specification
polykernel_degree_vec = [1, 2, 4, 6, 8]
n_misp=len(polykernel_degree_vec)

#Task loss mixture weights
n_wghts = 4
mixture_weights = np.array([0.2,0.4,0.6,0.8])

#number of repetitions of each trial
n_reps = 8

n_test = 10000
n_holdout = 500

#### Regressor

In [6]:
# initialize choice of regressor
regressor = LinearRegression
random_regr=False

In [12]:
results_df = pd.DataFrame()

oracle = ShortestPathOracle(graph_params)


for misspec_ind, degree in enumerate(polykernel_degree_vec): 
    
    polykernel_degree = degree
    data_params = [1000, n_test, n_holdout, polykernel_degree,polykernel_noise_half_width,B_true]
    [X_train, c_train,X_validation, c_validation, X_test, c_test] = generate_data(1000, 
            n_test, n_holdout, polykernel_degree,polykernel_noise_half_width,B_true,gen_test=True)
        
    testDict = generateInstanceDict(X_test, c_test, oracle)

    for ind_n, en_train in enumerate(N_s): 
        print('n ,', en_train)
        data_params = [en_train, n_test, n_holdout, polykernel_degree,polykernel_noise_half_width,B_true]

        # Each replication returns [original_regrets, reweighted_mean_regrets (n_wghts) ]
        res = Parallel(n_jobs=-1, verbose=20)(delayed(run_replication_over_weights)(data_params, X_test, c_test, testDict, 
                                 mixture_weights, regressor, graph_params, num_reweights = 3, random_regr=False) for k in range(n_reps))
        
        res_new =  pd.DataFrame.from_records(np.array(res).flatten())
        results_df = results_df.append(res_new)
        results_df.to_csv('results/may19_exp_3.csv')

n , 100


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed:  8.5min remaining: 25.4min
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:  8.5min remaining: 14.1min
[Parallel(n_jobs=-1)]: Done   4 out of   8 | elapsed:  8.5min remaining:  8.5min
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed:  8.6min remaining:  5.2min
[Parallel(n_jobs=-1)]: Done   6 out of   8 | elapsed:  8.7min remaining:  2.9min
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:  8.7min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:  8.7min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


n , 1000


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 17.8min
[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed: 17.9min remaining: 53.7min
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed: 18.2min remaining: 30.3min
[Parallel(n_jobs=-1)]: Done   4 out of   8 | elapsed: 18.3min remaining: 18.3min
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed: 18.5min remaining: 11.1min
[Parallel(n_jobs=-1)]: Done   6 out of   8 | elapsed: 18.5min remaining:  6.2min
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 18.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 18.6min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


n , 2000


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed:  7.0min remaining: 21.0min
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:  7.1min remaining: 11.9min
[Parallel(n_jobs=-1)]: Done   4 out of   8 | elapsed:  7.1min remaining:  7.1min
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed:  7.1min remaining:  4.3min
[Parallel(n_jobs=-1)]: Done   6 out of   8 | elapsed:  7.2min remaining:  2.4min
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:  7.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:  7.2min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


n , 1000


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 11.4min
[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed: 11.4min remaining: 34.2min
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed: 11.6min remaining: 19.3min
[Parallel(n_jobs=-1)]: Done   4 out of   8 | elapsed: 11.6min remaining: 11.6min
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed: 11.6min remaining:  7.0min
[Parallel(n_jobs=-1)]: Done   6 out of   8 | elapsed: 11.6min remaining:  3.9min
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 11.7min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 11.7min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


n , 2000


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 16.5min
[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed: 16.5min remaining: 49.4min
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed: 16.6min remaining: 27.7min
[Parallel(n_jobs=-1)]: Done   4 out of   8 | elapsed: 16.7min remaining: 16.7min
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed: 16.7min remaining: 10.0min
[Parallel(n_jobs=-1)]: Done   6 out of   8 | elapsed: 16.7min remaining:  5.6min
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 16.8min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 16.8min finished
